In [2]:
from collections import defaultdict
import os
from itertools import combinations,groupby

In [3]:
file_name = "all"   # this line need to change when dealing with different dataset
path = '/Users/yidesdo21/Projects/outputs/12_time_slicing/ptc_results/'

ptc_dict = defaultdict(set)
ptc_contents = list()
pubtator_files = os.listdir(path)
categories = ['CellLine','Chemical','Chromosome','DNAAcidChange','DNAMutation','Disease','Gene','ProteinAcidChange','ProteinMutation','RefSeq','SNP','Species']

for pubtator_file in pubtator_files:
    if pubtator_file.startswith(file_name) and pubtator_file.endswith(".PubTator"):
        with open(path+pubtator_file) as f:
            pmid_results = f.read().replace("\t", " ").split("\n")

            # group each article and corresponding annotations by using the split ''
            pmid_groups = (list(g) for _, g in groupby(pmid_results, key=''.__ne__))
            pmid_content = [a + b for a, b in zip(pmid_groups, pmid_groups)]

            ptc_contents.extend(pmid_content)


In [4]:
# Goal 01: create uid -- mention dictionary 
# Goal 02: create uid -- category dictionary
ptc_mention_uid = defaultdict(set)
ptc_category_uid = defaultdict(set)

for p in ptc_contents:
    # annotation output format: "article id" "starting index" "ending index" "annotated text" "category" "uid"
    if len(p) > 3:   # exclude the articles when the length of the list is three, because they have no annotations
        annos = p[2:-1]
        
        # loop for the annotations
        
        for anno in annos:
            anno_split = anno.split(" ")

        # sometimes annotation is blank [""], it is not valid annotations,
        #      the blank annotation is ignored.
            if len(anno_split) > 1:
            # if the second to last index is not in the category, 
            #      then it means the UID might not be in the normal format.   
#                 print(anno_split)
            
            
                if anno_split[-2] not in categories:
                    # change the category and uid index
                    mention, category, uid = anno_split[-4], anno_split[-3], anno_split[-2]+" "+anno_split[-1]
                else: 
                    mention, category, uid = anno_split[-3], anno_split[-2], anno_split[-1]
                    
#                 if category == "Chromosome":
#                     print(anno_split)
#                     print(p)
#                     print("-------")
                
                if uid not in ["-", ""]:
                    if not (category in ['Gene'] and len(uid.split(";")) > 1):
                        article_id = anno_split[0]   # this line will be run multiple times, but ignore this now
                        ptc_mention_uid[uid].add(mention)
                        ptc_category_uid[uid].add(category)
                        ptc_dict["NLM"+"_"+article_id].add(uid)

In [11]:
ptc_mention_uid.get("10090")

{'129Sv',
 '129sv',
 'BV',
 'Mice',
 'Mouse',
 'Murine',
 'Transgenic',
 'mice',
 'mouse',
 'murine',
 'subspecies',
 'transgenic',
 'transgenic-mice'}

In [10]:
ptc_category_uid.get("10090")

{'Species'}

In [4]:
# no need to preprocess it, it only means 12721 can be Gene, and also be Species
#.  so the identifier 12721 should be viewed as double
# some uids have the same number, but they are actually different
# bc. they belong to different categories/types, the ptc doesn't sign the differences for these uids 
for k,v in ptc_category_uid.items():
    if len(v) > 1:
        print(k,v)

12721 {'Gene', 'Species'}
837 {'Gene', 'Species'}
20 {'Gene', 'Chromosome'}
12 {'Gene', 'Chromosome'}
4217 {'Gene', 'Species'}
19 {'Gene', 'Chromosome'}
5341 {'Gene', 'Species'}
9 {'Gene', 'Chromosome'}
2 {'Gene', 'Chromosome'}
470 {'Gene', 'Species'}
4513 {'Gene', 'Species'}
9520 {'Gene', 'Species'}
11034 {'Gene', 'Species'}
11909 {'Gene', 'Species'}
3567 {'Gene', 'Species'}
4565 {'Gene', 'Species'}
3627 {'Gene', 'Species'}
1613 {'Gene', 'Species'}
50507 {'Gene', 'Species'}
1282 {'Gene', 'Species'}
18 {'Gene', 'Chromosome'}
5476 {'Gene', 'Species'}


In [ ]:
## needs further preprocessing when applying the types to the texts
## see in raw_data_and_evaluation.ipynb

### codes copied from that notebook
# semantic_nodes = defaultdict(list)
# sem_node_cnt = OrderedDict()

# for node in list(accum_v)[1:]:
#     cate = uid_category.get(node)
    
#     # gives None results
#     if node not in ['http://www.ifomis.org/bfo/1.1#Entity', 'rdfs:label', 'obo:IDO_0000666']:
#         # corner cases:
#         if cate in ["ProteinMutation'}}","ProteinMutation","DNAMutation","ProteinAcidChange","DNAAcidChange"]:
#             semantic_nodes["Mutation"].append(node)

#         elif cate == "Gene', 'Species":  # it means the identifer appear both in Gene and Species types, so have to count twice
#             semantic_nodes["Gene"].append(node)
#             semantic_nodes["Species"].append(node)

#         elif cate in ["Gene', 'Chromosome", "Chromosome"]:
#             semantic_nodes["Gene"].append(node)
#     #         semantic_nodes["Chromosome"].append(node)

#         elif cate == "RefSeq":
#             semantic_nodes["Mutation"].append(node)

#         else:
#             semantic_nodes[cate].append(node)

# for k,v in semantic_nodes.items():
#     sem_node_cnt[k] = len(v)
    


### a list of uids that relate to amyloid, we filter them 
- 351, amyloid beta precursor protein [Homo sapiens (human)], Gene type,
- 11820, include, amyloid beta (A4) precursor protein [Mus musculus (house mouse)], belongs to Gene type

- MESH:D016657, exclude, Cerebral Amyloid Angiopathy, a pathology of AD, mentions include 'beta-amyloid'. But it is viewed as a Disease. Have 21 mentions in the testing feature network. I am not sure. Maybe we should exclude it. I'll exclude it because 1) 11820 and 351 have more than 1k mentions, and 2) this uid belongs to Disease type. 
- MESH:D000686, exclude, amyloidosis is a disorder
- MESH:D009101, exclude, Multiple Myeloma is a disease
- MESH:D000544, exclude, Alzheimer Disease
- MESH:D003072, exclude, Cognition Disorders
- MESH:D007859, exclude, Learning Disabilities


In [7]:
## task: find the uids that contain the β-amyloid plaques
cnt = 0

for k,vs in ptc_mention_uid.items():
    vs_join = " ".join(vs)
    if "amyloid" in vs_join:
        print(k,vs)
        cnt += 1
        print("------------")
        
print(cnt)

MESH:D000686 {'amyloidosis', 'disorders', 'Inflammation', 'beta-amyloidosis', 'Amyloidosis', 'intracerebroventricular', 'astrogliosis', 'amyloidoses', 'beta-Amyloidosis', 'impairs', 'diseases', 'Abeta-amyloidosis', 'disease', 'amyloidosis-beta'}
------------
MESH:D009101 {'amyloidosis', 'myeloma', 'Myeloma'}
------------
MESH:D000544 {'ADNI', 'Changes', 'plaque', 'Fibrils', 'neurodegeneration', 'Coordinating', 'amyloidogenesis', "Alzheimer's-disease-linked", "Alzheimer's", 'hypothesis', 'neuritic', 'syndromes', 'DISEASE', 'deposits', 'aggregation', 'Society', 'subunits', 'Initiative', 'cognitive', 'toxicity', 'beta-Amyloidosis', 'disease', 'type', '(AD)dementia', 'beta-amyloidosis', 'syndrome', 'Defects', "Alzheimer'sdisease", 'Disease', 'sites', 'Pathology', 'Disorders', 'diseAse', 'Brain', 'deaths', 'deposition', 'lesions', 'leptomeninges', 'toxic', 'deficient', 'fAD', 'deficits', 'etiopathogenesis', 'ADD', 'accumulates', 'Parkinson', 'amyloidosis', 'Alzheimer', 'plaques', 'Composite

In [11]:
ptc_mention_uid.get("CVCL_0060;NCBITaxID:9606")

{'H1299'}